In [1]:
import xgboost as xgb
import pandas as pd
from sklearn import model_selection
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import RandomForestClassifier
from mlxtend.classifier import StackingCVClassifier
from sklearn.ensemble import GradientBoostingClassifier
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier

C:\Users\Rohit\Anaconda3\envs\iim\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
from sklearn.preprocessing import LabelEncoder
number = LabelEncoder()

In [3]:
clf1 = LogisticRegression()
clf2 = RandomForestClassifier(n_estimators=2000)
clf3 = GradientBoostingClassifier(learning_rate=0.1, loss='deviance', max_depth=5,n_estimators=2000,subsample=.5)
clf4 = AdaBoostClassifier(n_estimators=1500,learning_rate=0.5)
clf5 = ExtraTreesClassifier(n_estimators =1500,max_depth=10,n_jobs=-1)
clf6 = KNeighborsClassifier(n_neighbors=8,weights='distance',n_jobs=-1)
clf7 = MLPClassifier(hidden_layer_sizes=(400,3),max_iter=1000)
clf8 = KNeighborsClassifier(n_neighbors=2,weights='distance',n_jobs=-1)
clf9 = KNeighborsClassifier(n_neighbors=4,weights='distance',n_jobs=-1)
clf10 = KNeighborsClassifier(n_neighbors=8,weights='distance',n_jobs=-1)
clf11 = KNeighborsClassifier(n_neighbors=16,weights='distance',n_jobs=-1)
clf12 = KNeighborsClassifier(n_neighbors=32,weights='distance',n_jobs=-1)
clf13 = KNeighborsClassifier(n_neighbors=64,weights='distance',n_jobs=-1)
clf14 = KNeighborsClassifier(n_neighbors=128,weights='distance',n_jobs=-1)

In [4]:
gb = xgb.XGBClassifier(learning_rate =0.08, n_estimators=1000, 
                       max_depth=15, min_child_weight=1,
                       gamma=2,subsample=0.4,
                       colsample_bytree=0.55,objective= 'multi:softprob',
                       nthread=4,scale_pos_weight=1,seed=27)

sclf = StackingCVClassifier(classifiers=[clf2, clf3, clf4, clf5, clf6, clf7, clf8, clf9,clf10, clf11, clf12, clf13, clf14,gb],
                            meta_classifier=gb)
sclf1 = StackingCVClassifier(classifiers=[clf2, clf3, clf4, clf5, clf6, clf7, clf8, clf9,clf10, clf11, clf12, clf13, clf14, gb],
                            meta_classifier=clf7)
sclf2 = StackingCVClassifier(classifiers=[clf2, clf3, clf4, clf5, clf6, clf7, clf8, clf9,clf10, clf11, clf12, clf13, clf14, gb],
                            meta_classifier=clf5)

In [5]:
from mlxtend.classifier import EnsembleVoteClassifier
eclf = EnsembleVoteClassifier(clfs=[sclf, sclf1, sclf2], weights=[1.5,.2,.1])

In [ ]:
train = pd.read_csv('train.csv')
train['card_offered'] = number.fit_transform(train['card_offered'].astype('str'))
y = train['card_offered'].ravel()
train = train.drop(['cust_key','score_affinity_spend','nbr_hist_elite_offer_extns','spend_retail','card_accepted','card_offered','mem_fee_24m','nbr_hist_supp_offer_extns'], axis=1)
train = pd.get_dummies(train)
x = train.values

In [ ]:
t1 =pd.read_csv('test1.csv')
t1 = t1.drop(['cust_key','score_affinity_spend','nbr_hist_elite_offer_extns','spend_retail','mem_fee_24m','nbr_hist_supp_offer_extns'], axis=1)
t1 = pd.get_dummies(t1)
testA = t1.values

In [ ]:
t2 =pd.read_csv('test2.csv')
t2 = t2.drop(['cust_key','score_affinity_spend','nbr_hist_elite_offer_extns','spend_retail','mem_fee_24m','nbr_hist_supp_offer_extns'], axis=1)
t2 = pd.get_dummies(t2)
testB = t2.values

# Stacking CV

In [ ]:
st = model_selection.cross_val_score(sclf, x, y, 
                                              cv=4, scoring='accuracy',n_jobs = -1) #Stacking model CV calculation
st.mean()

In [ ]:
st1 = model_selection.cross_val_score(sclf1, x, y, 
                                              cv=4, scoring='accuracy',n_jobs = -1) #Stacking model CV calculation
st1.mean()

In [ ]:
st2 = model_selection.cross_val_score(sclf2, x, y, 
                                              cv=2, scoring='accuracy',n_jobs = -1) #Stacking model CV calculation
st2.mean()

# Ensemble 

In [ ]:
es = model_selection.cross_val_score(eclf, x, y, 
                                              cv=2, scoring='accuracy',n_jobs = -1) #Ensemble model CV Calculation
es.mean()

# Fitting the model

In [ ]:
model1 = eclf.fit(x,y)

In [ ]:
predictsA=model1.predict(testA)
i1 = pd.DataFrame({ 'card_offered': predictsA })

In [ ]:
i1.card_offered.value_counts()

In [ ]:
predictsB=model1.predict(testB)
i2 = pd.DataFrame({ 'card_offered': predictsB })

In [ ]:
i1.to_csv("s1.csv", index=False)

In [ ]:
i2.to_csv("s2.csv", index=False) #2- Supp 1-Elite 0-Credit

# Second GO

In [6]:
i1 = pd.read_csv("s1.csv")

In [7]:
i2 = pd.read_csv("s2.csv")

In [8]:
t11 = pd.read_csv("test1.csv")
t11 = pd.concat([t11, i1], axis=1)
t11 = t11.drop(['cust_key','score_affinity_spend','nbr_hist_elite_offer_extns','spend_retail','mem_fee_24m','nbr_hist_supp_offer_extns'], axis=1)
t11 = pd.get_dummies(t11)
testA1 = t11.values

In [9]:
t12 = pd.read_csv("test2.csv")
t12 = pd.concat([t12, i2], axis=1)
t12 = t12.drop(['cust_key','score_affinity_spend','nbr_hist_elite_offer_extns','spend_retail','mem_fee_24m','nbr_hist_supp_offer_extns'], axis=1)
t12 = pd.get_dummies(t12)
testB1 = t12.values

In [10]:
train1 = pd.read_csv("train.csv")
train1['card_accepted'] = number.fit_transform(train1['card_accepted'].astype('str'))
yr = train1['card_accepted'].ravel()
train1 = train1.drop(['cust_key','score_affinity_spend','nbr_hist_elite_offer_extns','spend_retail','mem_fee_24m','nbr_hist_supp_offer_extns','card_accepted'], axis=1)
train1 = pd.get_dummies(train1)
trainfinal = train1.values  

# Fitting the model

In [ ]:
model2 = sclf.fit(trainfinal,yr)

In [ ]:
predictsA1 = model2.predict(testA1)
p1 = pd.DataFrame({ 'card_offered': predictsA1 })
p1.to_csv("sub1.csv", index=False)
p1.card_offered.value_counts()

In [ ]:
predictsB1 = model2.predict(testB1)
p2 = pd.DataFrame({ 'card_offered': predictsB1 })
p2.to_csv("sub2.csv", index=False)  #2-None 1-Elite 3-Supp 0-Credit